**Implementación en Python**

**Data set utilizado:** MovieLens 1M Dataset (disponible en https://grouplens.org/datasets/movielens/)

**Fuente original de este código:** https://beckernick.github.io/matrix-factorization-recommender/. A continuación, una adaptación.

In [2]:
import pandas as pd
import numpy as np
import os

**Cargamos los datos: una lista de las películas y la matriz de ratings**

In [3]:
ratings_list = [i.strip().split("::") for i in open('ratings.dat', 'r').readlines()]
movies_list = [i.strip().split("::") for i in open('movies.dat', 'r', encoding='UTF-8', errors='ignore').readlines()]

ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])

In [191]:
movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [192]:
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


**Conviertimos UserID y MovieID de string a numeric en ambos dataframes, pues más adelante lo vamos a necesitar para poder ejecutar un .merge() de los dos datasets**

In [193]:
movies_df['MovieID'] = pd.to_numeric(movies_df['MovieID'])

In [194]:
ratings_df['MovieID'] = pd.to_numeric(ratings_df['MovieID'])

In [195]:
ratings_df['UserID'] = pd.to_numeric(ratings_df['UserID'])

**Contamos con 6040 usuarios únicos**

In [196]:
ratings_df['UserID'].nunique()

6040

**En total, han reankeado 3706 películas únicas**

In [197]:
ratings_df['MovieID'].nunique()

3706

**Convertimos la matriz de ratings en una que tenga los usuarios en filas y las películas en columnas**

In [198]:
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
R_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**En total tenemos, 6040 filas y 3706 columnas**

In [199]:
R_df.shape

(6040, 3706)

**Para poder utilizar esta matriz en el recomendador necesitamos 2.convertir el data frame en un numpy array y 2. normalizar por la media de cada usuario**

In [200]:
R = R_df.to_numpy()
R = R.astype(np.float)
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

**A continuación, descomponemos la matriz de ratings**

In [201]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(sigma)

**Usaremos su producto matricial para predecir la matriz de ratings completa con la que, finalmente, haremos las recomendaciones**

In [202]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [203]:
preds_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.289,0.143,-0.195,-0.019,0.012,-0.177,-0.074,0.141,-0.060,-0.196,...,0.028,0.002,0.026,-0.022,-0.085,0.404,0.106,0.032,0.050,0.089
1,0.745,0.170,0.335,0.001,0.022,1.353,0.051,0.071,0.162,1.567,...,-0.057,-0.014,-0.011,0.063,-0.016,0.156,-0.419,-0.101,-0.054,-0.140
2,1.819,0.456,0.091,-0.043,-0.026,-0.159,-0.132,0.099,0.031,0.735,...,0.040,-0.005,0.013,0.029,0.021,0.122,0.076,0.012,0.015,-0.110
3,0.408,-0.073,0.040,0.089,0.042,0.238,-0.049,0.009,0.045,-0.111,...,0.009,-0.005,-0.009,-0.003,-0.084,0.095,0.058,-0.026,0.015,-0.034
4,1.574,0.021,-0.051,0.247,-0.032,1.552,-0.200,-0.015,-0.060,0.451,...,0.110,0.046,0.007,-0.016,-0.050,-0.053,0.507,0.034,0.126,0.199


**Creamos la función que, consumiendo la matriz de ratings predichos, recomienda a un usuario las películas que más podrían gustarle y aún no vio**

In [204]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Obtenemos los ratings predichos para el usuario en cuestión y ordenamos sus ratings de mayor a menor
    user_row_number = userID - 1 # UserID comienza en 1, no 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # De la información original de este usuario, obtenemos cuántas y cuáles películas ya vio
    user_data = original_ratings_df[original_ratings_df['UserID'] == userID]
    user_full = user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').sort_values(['Rating'], ascending=False)
    
    # Recomendamos las primeras que no vio con mayor rating predicho
    sorted_user_predictions=pd.DataFrame(sorted_user_predictions).reset_index()
    sorted_user_predictions['MovieID'] = pd.to_numeric(sorted_user_predictions['MovieID'])

    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].merge(sorted_user_predictions, how = 'left',left_on = 'MovieID',right_on = 'MovieID').
    rename(columns = {user_row_number: 'Predictions'}).
    sort_values('Predictions', ascending = False).
    iloc[:num_recommendations, :-1])

    return user_full, recommendations

**Corremos la función de recomendación para algún UserID**

In [205]:
already_rated, predictions = recommend_movies(preds_df, 2, movies_df, ratings_df, 10)

In [206]:
already_rated.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres
0,2,1357,5,978298709,Shine (1996),Drama|Romance
28,2,2236,5,978299220,Simon Birch (1998),Drama
33,2,3147,5,978298652,"Green Mile, The (1999)",Drama|Thriller
35,2,1293,5,978298261,Gandhi (1982),Drama
40,2,110,5,978298625,Braveheart (1995),Action|Drama|War


In [210]:
predictions.head(10)

,MovieID,Title,Genres
1477,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
699,733,"Rock, The (1996)",Action|Adventure|Thriller
503,527,Schindler's List (1993),Drama|War
1815,1961,Rain Man (1988),Drama
1590,1704,Good Will Hunting (1997),Drama
361,377,Speed (1994),Action|Romance|Thriller
251,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1048,1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1503,1608,Air Force One (1997),Action|Thriller
453,474,In the Line of Fire (1993),Action|Thriller
